In [ ]:
import sys
from pathlib import Path    # handle paths to files 
from tqdm import tqdm        # progress bar for long loops

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.io import ascii, fits
from astropy.table import Table, vstack
from astropy.nddata import NDData, StdDevUncertainty, Cutout2D     # useful when working with data
from astropy.wcs import WCS                                        # astronomical coordinates systems
from astropy.coordinates import SkyCoord                           # same

import pyneb                                                          # emission line stuff (used for extinction correction)
from reproject import reproject_exact, reproject_interp               # reproject images
from skimage.measure import find_contours                             # find contours of mask
from regions import PixCoord, PolygonPixelRegion, CircleSkyRegion     # handle the regions

In [ ]:
# choose which galaxy and cluster classification file to use

gal_name = 'NGC3351' #'NGC1433' #3351'
version  = 'v1p1'
internal_release='IR2'

# the folder with the data (structure similar to Google Drive)
data_ext = Path('.')/'Archive'

In [ ]:
##### HST
hst_gal_name = f'{gal_name.lower()}'
print(f'reading HST data for {hst_gal_name}')

folder= data_ext/'Products'/'cluster_catalogs'/f'{version}'/f'{internal_release}'

if gal_name=='NGC0628':

    ## ngc0628c
    filename=folder/f'{hst_gal_name}c_phangshst_{version}_candidates_bcw_{internal_release}.fits'
    cluster_c= Table(fits.getdata(filename,1))
    
    ## ngc0628e
    filename=folder/f'{hst_gal_name}e_phangshst_{version}_candidates_bcw_{internal_release}.fits'
    cluster_e= Table(fits.getdata(filename,1))
    
    clusters=vstack([cluster_c, cluster_e])
    
else:
    filename = folder/f'{hst_gal_name}_phangshst_{version}_candidates_bcw_{internal_release}.fits'
    clusters=Table(fits.getdata(filename,1))
    
# modify table (rename the columns such that the clusters and associations are identical)
clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.degree,clusters['PHANGS_DEC']*u.degree)   

In [ ]:
##### MUSE 
## Read Ηα and Hβ from MUSE 
print(f'reading MUSE data for {gal_name}')
muse_filename = data_ext /'MUSE'/'DR2.1'/'MUSEDAP'/f'{gal_name}_MAPS.fits'

with fits.open(muse_filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    Hbeta = NDData(data=hdul['HB4861_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HB4861_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HB4861_FLUX'].data),
                    meta=hdul['HB4861_FLUX'].header,
                    wcs=WCS(hdul['HB4861_FLUX'].header))

In [ ]:
##### get Halpha and Hbeta fluxes at each cluster location

# the output table with columns for the Halpha/Hbeta fluxes
tbl = clusters.copy()#[['ID_PHANGS_CLUSTERS','PHANGS_X','PHANGS_Y']]
tbl.add_column(gal_name, index=0, name='gal_name')
tbl['HA6562_FLUX'] = np.nan
tbl['HB4861_FLUX'] = np.nan
tbl['HA6562_FLUX_ERR'] = np.nan
tbl['HB4861_FLUX_ERR'] = np.nan

# empty list of region for plotting 
cluster_region_dict={k:{'classification':v} for k,v in clusters['ID_PHANGS_CLUSTERS', 'PHANGS_CLUSTER_CLASS_HUMAN']}

for row in tqdm(tbl):
    
    # it is way faster to search for the regions in a cutout
    cluster_ID = row['ID_PHANGS_CLUSTERS']
    position = clusters[clusters['ID_PHANGS_CLUSTERS']==cluster_ID]['SkyCoord'][0]
    
    try:
        # sometimes the associations are outside of the MUSE FOV
        cutout_Halpha=Cutout2D(Halpha.data, position, size=20*u.arcsec, wcs=Halpha.wcs)
        cutout_Hbeta=Cutout2D(Hbeta.data,position, size=20*u.arcsec,wcs=Hbeta.wcs)
        cutout_Halpha_err=Cutout2D(Halpha.uncertainty.array,position,size=20*u.arcsec,wcs=Halpha.wcs)
        cutout_Hbeta_err=Cutout2D(Hbeta.uncertainty.array,position,size=20*u.arcsec,wcs=Hbeta.wcs)
    except:
        continue
        
    # in rare cases this will return multiple regions. we are lazy and just use the first one
    reg_sky=CircleSkyRegion(position, (0.4/3600)*u.deg)
    reg_pix_muse = reg_sky.to_pixel(cutout_Halpha.wcs)
    mask = reg_pix_muse.to_mask(mode='subpixels',subpixels=16)
    
    ## make list of regions for plotting
    cluster_region_dict[cluster_ID]['region']=reg_sky
            
    try:
        # sum up the flux inside of this mask
        row['HA6562_FLUX'] = np.sum(mask.multiply(cutout_Halpha.data))
        row['HB4861_FLUX'] = np.sum(mask.multiply(cutout_Hbeta.data))

        # for the uncertainty we take the square root of the sum of the squared uncertainties
        row['HA6562_FLUX_ERR'] = np.sqrt(np.sum(mask.multiply(cutout_Halpha.data)**2))
        row['HB4861_FLUX_ERR'] = np.sqrt(np.sum(mask.multiply(cutout_Hbeta.data)**2))
    except:
        print(f'{cluster_ID}\tBAD')
        continue

In [ ]:
# Milky Way E(B-V) from  Schlafly & Finkbeiner (2011)
EBV_MW = {'IC5332': 0.015,'NGC0628': 0.062,'NGC1087': 0.03,'NGC1300': 0.026,
          'NGC1365': 0.018,'NGC1385': 0.018,'NGC1433': 0.008,'NGC1512': 0.009,
          'NGC1566': 0.008,'NGC1672': 0.021,'NGC2835': 0.089,'NGC3351': 0.024,
          'NGC3627': 0.037,'NGC4254': 0.035,'NGC4303': 0.02,'NGC4321': 0.023,
          'NGC4535': 0.017,'NGC5068': 0.091,'NGC7496': 0.008}

In [ ]:
#### Correct for extinction 

print('correct for extinction')
# Milky Way extinction
rc_MW = pyneb.RedCorr(E_BV = EBV_MW[gal_name], R_V = 3.1, law = 'CCM89 oD94')

tbl['HA6562_FLUX'] *= rc_MW.getCorr(6562) 
tbl['HB4861_FLUX'] *= rc_MW.getCorr(4861)
tbl['HA6562_FLUX_ERR'] *= rc_MW.getCorr(6562) 
tbl['HB4861_FLUX_ERR'] *= rc_MW.getCorr(4861)

# Internal extinction is estimated from the Balmer decrement
rc_balmer = pyneb.RedCorr(R_V = 3.1, law = 'CCM89 oD94')
rc_balmer.setCorr(obs_over_theo= tbl['HA6562_FLUX']/tbl['HB4861_FLUX'] / 2.86, wave1=6562.81, wave2=4861.33)
# set E(B-V) to zero if S/N is less than 3 in Halpha or Hbeta
rc_balmer.E_BV[(rc_balmer.E_BV<0) | (tbl['HB4861_FLUX']<3*tbl['HB4861_FLUX_ERR']) |  (tbl['HA6562_FLUX']<3*tbl['HA6562_FLUX_ERR'])] = 0
tbl['EBV_balmer'] = rc_balmer.E_BV
#tbl['A5007'] =  -2.5*np.log10(rc.getCorr(5007))

tbl['HA6562_FLUX_CORR'] = tbl['HA6562_FLUX'] * rc_balmer.getCorr(6562) 
tbl['HB4861_FLUX_CORR'] = tbl['HB4861_FLUX'] * rc_balmer.getCorr(4861)
tbl['HA6562_FLUX_CORR_ERR'] = tbl['HA6562_FLUX_ERR'] * rc_balmer.getCorr(6562) 
tbl['HB4861_FLUX_CORR_ERR'] = tbl['HB4861_FLUX_ERR'] * rc_balmer.getCorr(4861)

In [ ]:
##### Save tbl to fits file

tbl.remove_column('SkyCoord')

print('write to file')
primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(tbl)
hdul = fits.HDUList([primary_hdu, table_hdu])

outputFile= f'/home/rebeccaminsley/phangs/Muse_Cluster_Data/{hst_gal_name}_phangshst_{version}_candidates_bcw_{internal_release}_Halpha.fits'
hdul.writeto(outputFile,overwrite=True)

### Plots

In [ ]:
%matplotlib widget
import seaborn as sns

In [ ]:
def get_cmapscaling(data, base=1000):
    array = ((data*base)+1)
    data_ = np.log(array)/np.log(base)
    return(data_)

def get_plot(nddata, scaling=99.9, hdu=None, base=1000, figsize=(8,6)):
    try: 
        data_ = get_cmapscaling(nddata.data)
    except: 
        data_ = get_cmapscaling(nddata)
    
    vmin = np.nanpercentile(data_[data_!=0], 100-scaling)
    vmax = np.nanpercentile(data_[data_!=0], scaling)
#     print(vmin, vmax)  
    try: 
        wcs = nddata.wcs#WCS(data.header)
        fig = plt.figure()
        ax = fig.add_subplot(projection=wcs)
    except:
        fig = plt.figure()
        if hdu != None: 
            wcs = WCS(hdu.header)
            ax = fig.add_subplot(projection=wcs)
        else: 
            ax = fig.add_subplot()
    im = ax.imshow(data_, origin='lower', cmap='magma', vmin=vmin, vmax=vmax, interpolation='nearest')
    return((fig, ax), (vmin, vmax))

In [ ]:
def plot_im(nddata, ax, scaling=99.9, base=1000):
    try:
        data_ = get_cmapscaling(nddata.data)
    except:
        data_ = get_cmapscaling(nddata)
        
    vmin = np.nanpercentile(data_[data_!=0], 100-scaling)
    vmax = np.nanpercentile(data_[data_!=0], scaling)
    
    im = ax.imshow(data_, origin='lower', cmap='magma', vmin=vmin, vmax=vmax, interpolation='nearest')

In [ ]:
%matplotlib widget 
from matplotlib.patches import Circle

region_colors=['#708090', '#6341c7', '#add8e6']

fig=plt.figure(figsize=(15,6), dpi=100)

# plot all clusers 
ax1=fig.add_subplot(1,2,1, projection=Halpha.wcs)
plot_im(Halpha, ax1)

for ii,idx in enumerate(clusters['ID_PHANGS_CLUSTERS']):
    try:
        cl_reg = cluster_region_dict[idx]['region']
        cl_class=cluster_region_dict[idx]['classification']

        color=region_colors[int(cl_class-1)]

        cl_reg.visual ={'color':color, 'linewidth':1}
        cl_reg.to_pixel(ax1.wcs).plot(ax=ax1)
    except:
        continue


# Plot non overlap 
ax2=fig.add_subplot(1,2,2, projection=Halpha.wcs, sharey=ax1, sharex=ax1)
plot_im(Halpha, ax2)

## plot only regions that dont overlap
separation = [clusters['SkyCoord'].separation(tmp).arcsec for tmp in clusters['SkyCoord']]
overlap = np.array([np.any(np.where((tmp>0) & (tmp<=0.4), True, False)) for tmp in separation])
#np.nonzero((np.array(separation) < 4) & (np.array(separation) >0))

for cluster_idx in clusters['ID_PHANGS_CLUSTERS'][~overlap]:
    try:
        
        cl_reg = cluster_region_dict[cluster_idx]['region']
        cl_reg.to_pixel(ax2.wcs).plot(ax=ax2)
    except:
        continue

In [ ]:
dataFrame=clusters.copy().to_pandas()
#dataFrame.loc[dataFrame.PHANGS_CLUSTER_CLASS_HUMAN>4, 'PHANGS_CLUSTER_CLASS_HUMAN'] = 4
dataFrame['log_age'] = np.log10(dataFrame['PHANGS_AGE_MINCHISQ'].to_numpy())
dataFrame['log_mass'] = np.log10(dataFrame['PHANGS_MASS_MINCHISQ'].to_numpy())
dataFrame['E_BV']=tbl['EBV_balmer']
dataFrame['log_halpha']=np.log10(tbl['HA6562_FLUX_CORR'])

In [ ]:
#sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_style("ticks")

sns.set_palette('GnBu')
splt = sns.jointplot(data=dataFrame, 
                     x="log_age", 
                     y='log_mass', #"E_BV", 
                     marginal_ticks=True,
                     space=0,
                     marginal_kws=dict(fill=False),
                     marker=".", 
                     s=120, hue='PHANGS_CLUSTER_CLASS_HUMAN', palette=region_colors, alpha=0.9, edgecolor='none')

ax=splt.figure.axes[0]
ax.get_legend().remove()
ax.legend(loc='best')

splt.ax_marg_y.spines['right'].set_visible('True')
splt.ax_marg_y.spines['top'].set_visible('True')

splt.ax_marg_x.spines['right'].set_visible('True')
splt.ax_marg_x.spines['top'].set_visible('True')

In [ ]:
#sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_style("ticks")

sns.set_palette('GnBu')
splt = sns.jointplot(data=dataFrame, 
                     x="log_age", 
                     y="E_BV", 
                     marginal_ticks=True,
                     space=0,
                     marginal_kws=dict(fill=False),
                     marker=".", 
                     s=120, hue='PHANGS_CLUSTER_CLASS_HUMAN', palette=region_colors, alpha=0.9, edgecolor='none')

ax=splt.figure.axes[0]
ax.get_legend().remove()
ax.legend(loc='best')

splt.ax_marg_y.spines['right'].set_visible('True')
splt.ax_marg_y.spines['top'].set_visible('True')

splt.ax_marg_x.spines['right'].set_visible('True')
splt.ax_marg_x.spines['top'].set_visible('True')

In [ ]:
#sns.set_style("darkgrid", {"axes.facecolor": ".9"})
old=dataFrame.PHANGS_AGE_MINCHISQ

sns.set_style("ticks")

sns.set_palette('GnBu')
splt = sns.jointplot(data=dataFrame.loc[dataFrame.PHANGS_AGE_MINCHISQ<10,:], 
                     x="PHANGS_EBV_MINCHISQ", 
                     y="E_BV", 
                     marginal_ticks=True,
                     space=0,
                     marginal_kws=dict(fill=False),
                     marker=".", 
                     s=120, hue='PHANGS_CLUSTER_CLASS_HUMAN', palette=region_colors, alpha=0.9, edgecolor='none')

ax=splt.figure.axes[0]
ax.get_legend().remove()
ax.legend(loc='best')

ax.set_xlabel('SED reddening')

splt.ax_marg_y.spines['right'].set_visible('True')
splt.ax_marg_y.spines['top'].set_visible('True')

splt.ax_marg_x.spines['right'].set_visible('True')
splt.ax_marg_x.spines['top'].set_visible('True')

new_x=np.arange(0,3, 0.05)
new_y=2*new_x
ax.plot(new_x,new_y)

In [ ]:
old=dataFrame.PHANGS_AGE_MINCHISQ

sns.set_style("ticks")

sns.set_palette('GnBu')
splt = sns.jointplot(data=dataFrame.loc[dataFrame.PHANGS_AGE_MINCHISQ<10,:], 
                     x="PHANGS_EBV_MINCHISQ", 
                     y="log_halpha", 
                     marginal_ticks=True,
                     space=0,
                     marginal_kws=dict(fill=False),
                     marker=".", 
                     s=120, hue='PHANGS_CLUSTER_CLASS_HUMAN', palette=region_colors, alpha=0.9, edgecolor='none')

ax=splt.figure.axes[0]
ax.get_legend().remove()
ax.legend(loc='best')

ax.set_xlabel('SED reddening')

splt.ax_marg_y.spines['right'].set_visible('True')
splt.ax_marg_y.spines['top'].set_visible('True')

splt.ax_marg_x.spines['right'].set_visible('True')
splt.ax_marg_x.spines['top'].set_visible('True')

In [ ]:
old=dataFrame.PHANGS_AGE_MINCHISQ

sns.set_style("ticks")

sns.set_palette('GnBu')
splt = sns.jointplot(data=dataFrame.loc[dataFrame.PHANGS_AGE_MINCHISQ<10,:], 
                     x="PHANGS_EBV_MINCHISQ", 
                     y="log_halpha", 
                     marginal_ticks=True,
                     space=0,
                     marginal_kws=dict(fill=False),
                     marker=".", 
                     s=120, hue='PHANGS_CLUSTER_CLASS_HUMAN', palette=region_colors, alpha=0.9, edgecolor='none')

ax=splt.figure.axes[0]
ax.get_legend().remove()
ax.legend(loc='best')

ax.set_xlabel('SED reddening')

splt.ax_marg_y.spines['right'].set_visible('True')
splt.ax_marg_y.spines['top'].set_visible('True')

splt.ax_marg_x.spines['right'].set_visible('True')
splt.ax_marg_x.spines['top'].set_visible('True')

In [ ]:
ax.get_xlim()

### Plot color-color

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "normal"
plt.rcParams["axes.labelweight"] = "normal"
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in" 
plt.rcParams["ytick.right"] = 'True'
plt.rcParams["xtick.top"] = 'True'

In [ ]:
model_file_name='/home/rebeccaminsley/phangs/models-block-0_SolarZ_SSP_fcov0.txt'

file=open(model_file_name,'r').read()
lines=[line.split('  ') for line in fl.split('\n') if line!='']
for r, ln in enumerate(lines):
    lines[r]=[v.strip() for v in ln if v!='']

data=np.array(lines[1:], dtype=float)
columns=lines[0]

fcov_0_solar_df = pd.DataFrame(data, columns=columns)
fcov_0_solar_df=fcov_0_solar_df

In [ ]:
### For color-color plot
X=clusters['PHANGS_F555W_VEGA_TOT']-clusters['PHANGS_F814W_VEGA_TOT']
Y=clusters['PHANGS_F336W_VEGA_TOT']-clusters['PHANGS_F438W_VEGA_TOT']

# Halpha and Hbeta from muse 
C= tbl['EBV_balmer']#tbl['HA6562_FLUX_CORR']/tbl['HB4861_FLUX_CORR']

###### 
clust_class=clusters['PHANGS_CLUSTER_CLASS_HUMAN']

mass=clusters['PHANGS_MASS_MINCHISQ']
low_mass= mass<5000

vmin,vmax= np.nanquantile(C.data, [.1, .99])
# norm= matplotlib.colors.LogNorm() #CenteredNorm(np.nanmedian(C.data)) #matplotlib.colors.Normalize(vmin=vmin, vmax=vmax, clip=False)
kwrg={'cmap':'cividis', 'alpha':0.7, 'vmin':vmin, 'vmax':vmax}#, 'norm':norm}


# #### ------------------------
fig = plt.figure(figsize=(14,4), dpi=100)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=None)

for col, clss in enumerate([1,2,3]):
    
    ax=fig.add_subplot(1,3,col+1)
    
#     cbool=(clust_class==clss)
#     ax.scatter(X[cbool], 
#                Y[cbool], 
#                c=C[cbool], 
#                marker='.', s=35,
#                **kwrg) 
    
#     # cbool= (c_class==clss) & low_mass
#     ax.scatter(X[cbool&low_mass], 
#                Y[cbool&low_mass], 
#                c=C[cbool&low_mass], 
#                marker='*', s=30, label=f'n low mass: {len(X[cbool&low_mass])} ',
#                **kwrg) 
    
#     ax.legend(loc='lower left', fontsize='small')
#     ax.set_ylim(ax.get_ylim()[::-1])
    
    if col==0: ax.set_ylabel('(F336W-F438W)')
    if col==2: ax.set_xlabel('(F555W-F814W)')
    
    ax.plot(fcov_0_solar_df['F555W_UVIS_CHIP2'] - fcov_0_solar_df['F814W_UVIS_CHIP2'],
            fcov_0_solar_df['F336W_UVIS_CHIP2'] - fcov_0_solar_df['F438W_UVIS_CHIP2'],
            color='salmon', linewidth=2.0,
            label='BC03 Stellar Population Model -- $\mathrm{Z_{\odot}~(0.02)~|~f_{cov} = 0~|~SSP}$')

    
# axes=fig.get_axes()
# cax= fig.colorbar(ax.collections[0], ax=fig.get_axes())


#     # fig.tight_layout()

In [ ]:
model_file_name='/home/rebeccaminsley/phangs/models-block-0_SolarZ_SSP_fcov0.txt'

file=open(model_file_name,'r').read()
lines=[line.split('  ') for line in fl.split('\n') if line!='']
for r, ln in enumerate(lines):
    lines[r]=[v.strip() for v in ln if v!='']

data=np.array(lines[1:], dtype=float)
columns=lines[0]

fcov_0_solar_df = pd.DataFrame(data, columns=columns)
fcov_0_solar_df=fcov_0_solar_df

target_filter_list_no_err = ['F275W', 'F336W', 'F438W', 'F555W', 'F814W']

\
            convert_mjy_vega(bc03_solar_z_0cov[k.split('_')[0]+'_flux_mJy'],
                             enh_header_df.loc[enh_header_df['filter'] == k.split('_')[0], 'zpAB'].item(),
                             enh_header_df.loc[enh_header_df['filter'] == k.split('_')[0], 'zpVEGA'].item())
# for k in target_filter_list_no_err:

#         """Convert mJy to Vega"""

#         bc03_solar_z_0cov[k.split('_')[0] + '_vega'] = \
#             convert_mjy_vega(bc03_solar_z_0cov[k.split('_')[0]+'_flux_mJy'],
#                              enh_header_df.loc[enh_header_df['filter'] == k.split('_')[0], 'zpAB'].item(),
#                              enh_header_df.loc[enh_header_df['filter'] == k.split('_')[0], 'zpVEGA'].item())

In [ ]:
fits.getheader('/home/rebeccaminsley/phangs/models-block-0.fits',1)

In [ ]:
len(lines)

In [ ]:
pd.DataFrame?

In [ ]:
np.array(lines2[:-1]).shape
lines

In [ ]:
fl.split('\n')[-1]